# Resolviendo Sudokus
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

Las reglas del Sudoku son simples y bien definidas: debés completar las celdas vacías de manera que cada fila, cada columna y cada una de las cajas de 3x3 contenga todos los números del 1 al 9, sin repeticiones. Un ejemplo sería el siguiente:

<div>
<img src="./sudoku_1.png" width="600"/>
</div>

En este caso, el sudoku de la izquierda está sin resolver, mientras que el de la derecha muestra la solución, con los números resaltados en azul. Es importante destacar que, si un sudoku está bien diseñado, siempre tiene una única solución.

# Implementando conceptos básicos

Basándonos en el trabajo de Peter Norvig](https://github.com/norvig/pytudes/blob/main/ipynb/Sudoku.ipynb), vamos a definir los conceptos clave del Sudoku:

- **Dígitos**: Son los números del `1` al `9`
- **Filas**: Por convención, las 9 filas están etiquetadas con las letras `'A'` a `'I'` (de arriba hacia abajo).
- **Columnas**: Por convención, las 9 columnas están etiquetadas con los números `'1'` a `'9'` (de izquierda a derecha).
- **Celda**: Se nombra mediante la combinación de la etiqueta de la fila y la columna. Por ejemplo, `'A9'` es la celda en la esquina superior derecha.
- **Celda fijas**: Son aquellas cuyos valores ya están definidos al inicio del juego.
- **Cajas**: Las 9 cajas son bloques de 3x3 dentro de la grilla (destacadas con líneas negras en el diagrama).
- **Unidad**: Una unidad puede ser una fila, una columna o una caja. Cada unidad consta de 9 celdas.

Ahora necesitamos extender algunas definiciones para poder resolver el Sudoku usando algoritmos de búsqueda local:

- **Estado**: Es el llenado completo de una grilla de Sudoku, usando los 9 dígitos, uno por celda.
- **Solución**: Una grilla es válida como solución si cada unidad contiene los 9 dígitos sin repeticiones, y todos los dígitos están correctamente ubicados según las reglas del Sudoku.
- **Vecinos**: Dado un estado A, se define como vecino cualquier otro estado que se obtenga mediante una de las siguientes condiciones:
    1. Intercambio de valores entre dos celdas dentro de la misma unidad, siempre que ninguna de las celdas involucradas sea fija.
    2. Cambio de valor en una única celda, siempre y cuando no sea una celda fija.

Veamos algunos ejemplos que nos ayudarán a entender mejor estas definiciones.

Este es un estado del Sudoku. Obsérvese que no es una solución válida:

<div>
<img src="./sudoku_3.png" width="300"/>
</div>

En cambio, este estado sí es **una solución** válida:

<div>
<img src="./sudoku_4.png" width="300"/>
</div>

Ahora, pasemos a un ejemplo de vecinos. Partimos del siguiente estado:

<div>
<img src="./sudoku_3.png" width="300"/>
</div>

Un vecino de este estado, según la condición 1 (intercambio de valores entre celdas dentro de la misma unidad, sin que ninguna sea fija), sería:

<div>
<img src="./sudoku_5.png" width="300"/>
</div>

Y un vecino, según la condición 2 (cambio de valor de una única celda no fija), sería:

<div>
<img src="./sudoku_6.png" width="300"/>
</div>

---
## Implementación en código de Sudoku

A continuación, crearemos en el archivo `sudoku_stuff.py` diversas funciones que nos permitirán resolver sudokus. Estas funciones incluirán:

- Inicialización de un estado: Para configurar la grilla de Sudoku al inicio del juego.
- Creación de vecinos: Para generar los estados vecinos a partir de uno dado, según las reglas que hemos definido.
- Verificación de solución: Para comprobar si un estado es una solución válida de Sudoku, es decir, si cumple con todas las restricciones del juego.

In [1]:
from sudoku_stuff import *

### Obtenemos las coordenadas de las celdas

Podemos obtener las coordenadas de todas las celdas utilizando la función `obtain_all_cells()`:

In [2]:
squares = obtain_all_cells()

squares[:10]

('A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1')

Si queremos obtener las coordenadas para el caso de un Sudoku de 2x2, por ejemplo:

<div>
<img src="./sudoku_2.png" width="300"/>
</div>

Podemos hacerlo indicando las coordenadas de cada celda mediante strings, como se muestra a continuación:

In [3]:
squares = obtain_all_cells(rows="ABCD", cols="1234")

squares

('A1',
 'A2',
 'A3',
 'A4',
 'B1',
 'B2',
 'B3',
 'B4',
 'C1',
 'C2',
 'C3',
 'C4',
 'D1',
 'D2',
 'D3',
 'D4')

### Obtenemos cuáles son las unidades

Además, si queremos obtener las unidades que conforman un Sudoku de 3x3, podemos usar la función `obtain_coordinates_of_units()`.

Esta función nos devolverá un diccionario con las coordenadas de las unidades (filas, columnas y cajas) que forman el Sudoku:

In [4]:
units_dict = obtain_coordinates_of_units()

Podemos acceder a las siguientes claves del diccionario:
- `"boxes"`: Las cajas. Cada caja es una tupla que contiene las coordenadas de las celdas que la integran.
- `"rows"`: Las filas. Cada fila es una lista con las coordenadas de sus celdas.
- `"columns"`: Las columnas. Cada columna es una lista con las coordenadas de sus celdas.
- `"units"`: Todas las unidades (filas, columnas y cajas), representadas como una lista de tuplas de coordenadas.

Veamos algunos ejemplos:

- La primera caja:

In [5]:
units_dict["boxes"][0]

('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3')

- La primera fila:

In [6]:
units_dict["rows"][0]

('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1')

- La primera columna:

In [7]:
units_dict["columns"][0]

('A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9')

### Estado de Sudoku

Un estado de Sudoku se puede representar mediante un diccionario, donde las claves son las coordenadas de las celdas y los valores son los números que deben almacenarse en esas celdas. Es importante recordar que algunas celdas contienen valores fijos que no se pueden modificar. Estos valores fijos también pueden guardarse en un diccionario separado.

Si usamos como ejemplo este Sudoku:

<div>
<img src="./sudoku_7.png" width="300"/>
</div>

El diccionario de **celdas fijas** sería el siguiente:


In [8]:
fixed_squares = {
    'A1': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A7': 9,
    'B1': 1, 'B2': 8, 'B3': 5, 'B6': 9, 'B7': 7,
    'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D2': 2, 'D5': 1, 'D8': 4, 'D9': 9,
    'E2': 4, 'E3': 9, 'E5': 5, 
    'F3': 1, "F4": 9, "F5": 8, "F7": 6, "F8": 7,
    'G1': 4, 'G2': 9, 'G5': 3, 'G9': 7, 
    'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H9': 6,
    'I8': 8,
}

Y la solución correspondiente sería:

<div>
<img src="./sudoku_4.png" width="300"/>
</div>

Representada con el siguiente diccionario, que en este caso es un estado **solución**:

In [9]:
solution = {
    'A1': 3, 'A2': 7, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 1, 'A7': 9, 'A8': 2, 'A9': 8,
    'B1': 1, 'B2': 8, 'B3': 5, 'B4': 4, 'B5': 2, 'B6': 9, 'B7': 7, 'B8': 6, 'B9': 3,
    'C1': 9, 'C2': 6, 'C3': 2, 'C4': 3, 'C5': 7, 'C6': 8, 'C7': 4, 'C8': 1, 'C9': 5,
    'D1': 8, 'D2': 2, 'D3': 7, 'D4': 6, 'D5': 1, 'D6': 3, 'D7': 5, 'D8': 4, 'D9': 9,
    'E1': 6, 'E2': 4, 'E3': 9, 'E4': 2, 'E5': 5, 'E6': 7, 'E7': 8, 'E8': 3, 'E9': 1,
    'F1': 5, 'F2': 3, 'F3': 1, 'F4': 9, 'F5': 8, 'F6': 4, 'F7': 6, 'F8': 7, 'F9': 2,
    'G1': 4, 'G2': 9, 'G3': 6, 'G4': 8, 'G5': 3, 'G6': 2, 'G7': 1, 'G8': 5, 'G9': 7,
    'H1': 2, 'H2': 1, 'H3': 8, 'H4': 7, 'H5': 4, 'H6': 5, 'H7': 3, 'H8': 9, 'H9': 6,
    'I1': 7, 'I2': 5, 'I3': 3, 'I4': 1, 'I5': 9, 'I6': 6, 'I7': 2, 'I8': 8, 'I9': 4,
}

Ver este diccionario directamente puede dificultar la comprensión visual del Sudoku, pero contamos con una función que facilita esta tarea: `print_state()`.

In [10]:
print_state(solution)

*---------+---------+---------*
| 3  7  4 | 5  6  1 | 9  2  8 |
| 1  8  5 | 4  2  9 | 7  6  3 |
| 9  6  2 | 3  7  8 | 4  1  5 |
*---------+---------+---------*
| 8  2  7 | 6  1  3 | 5  4  9 |
| 6  4  9 | 2  5  7 | 8  3  1 |
| 5  3  1 | 9  8  4 | 6  7  2 |
*---------+---------+---------*
| 4  9  6 | 8  3  2 | 1  5  7 |
| 2  1  8 | 7  4  5 | 3  9  6 |
| 7  5  3 | 1  9  6 | 2  8  4 |
*---------+---------+---------*


### Obteniendo estados al azar

Una estrategia que nos ayudará a resolver estos Sudokus utilizando algoritmos de búsqueda local es inicializar el Sudoku en un estado aleatorio. Esto se puede lograr mediante la función `init_state()`, a la que debemos pasarle las celdas fijas.

In [11]:
new_state = init_state(fixed_squares)

In [12]:
print_state(new_state)

*---------+---------+---------*
| 3  4  4 | 5  6  4 | 9  7  9 |
| 1  8  5 | 1  9  9 | 7  7  2 |
| 2  2  3 | 5  7  8 | 4  1  5 |
*---------+---------+---------*
| 7  2  5 | 9  1  4 | 2  4  9 |
| 2  4  9 | 5  5  2 | 5  6  9 |
| 9  3  1 | 9  8  7 | 6  7  5 |
*---------+---------+---------*
| 4  9  7 | 4  3  1 | 3  4  7 |
| 8  1  8 | 7  4  5 | 3  9  6 |
| 2  8  3 | 4  6  6 | 9  8  4 |
*---------+---------+---------*


Obsérvese que este estado aleatorio no es una **solución válid** (aunque por pura casualidad podría serlo, es muy poco probable que ocurra).

### Verificando si es solución

Podemos verificar si un estado del Sudoku es una solución válida utilizando la función `is_solution()`. Esta función comprueba que todas las unidades (filas, columnas y cajas) contengan todos los dígitos del 1 al 9, sin repeticiones.

La función retorna un valor booleano, que podemos usar para verificar si el estado es una solución.

In [13]:
print("¿El estado obtenido es solución?")
if is_solution(new_state):
    print("¡Sí, es solución!")
else:
    print("No, no es solución.")

¿El estado obtenido es solución?
No, no es solución.


Verifiquemos con la solución real:

In [14]:
print("¿El estado definido como solution es realmente la solución?")
if is_solution(solution):
    print("¡Sí, es solución!")
else:
    print("No, no es solución.")

¿El estado definido como solution es realmente la solución?
¡Sí, es solución!


### Generando vecinos

Por último, tenemos una función que genera los vecinos inmediatos de un estado dado, basándose en las dos condiciones que vimos previamente:
1. Se intercambia el valor de una celda con otra dentro de la misma unidad, siempre que ninguna de las celdas involucradas sea fija.
2. También se consideran vecinos aquellos estados en los que el valor de una sola celda cambia, siempre y cuando esa celda no sea fija.

La función `return_neib_states()` obtiene todos estos vecinos. Para funcionar, necesita como entrada el estado del cual queremos generar los vecinos, así como las celdas fijas. Nos devuelve una lista con todos los nuevos estados vecinos.

In [15]:
neib = return_neib_states(new_state, fixed_squares)

In [16]:
print(f"Tenemos {len(neib)} vecinos")

Tenemos 605 vecinos


Veamos algunos ejemplos:

In [17]:
print_state(neib[0]) 

*---------+---------+---------*
| 3  4  4 | 5  6  4 | 9  7  9 |
| 1  8  5 | 1  9  9 | 7  7  2 |
| 7  2  3 | 5  7  8 | 4  1  5 |
*---------+---------+---------*
| 2  2  5 | 9  1  4 | 2  4  9 |
| 2  4  9 | 5  5  2 | 5  6  9 |
| 9  3  1 | 9  8  7 | 6  7  5 |
*---------+---------+---------*
| 4  9  7 | 4  3  1 | 3  4  7 |
| 8  1  8 | 7  4  5 | 3  9  6 |
| 2  8  3 | 4  6  6 | 9  8  4 |
*---------+---------+---------*


In [18]:
print_state(neib[25]) 

*---------+---------+---------*
| 3  4  4 | 5  6  4 | 9  7  9 |
| 1  8  5 | 1  9  9 | 7  7  2 |
| 2  2  3 | 5  7  8 | 4  1  5 |
*---------+---------+---------*
| 7  2  3 | 9  1  4 | 2  4  9 |
| 2  4  9 | 5  5  2 | 5  6  9 |
| 9  3  1 | 9  8  7 | 6  7  5 |
*---------+---------+---------*
| 4  9  7 | 4  3  1 | 3  4  7 |
| 8  1  8 | 7  4  5 | 3  9  6 |
| 2  8  5 | 4  6  6 | 9  8  4 |
*---------+---------+---------*


In [19]:
print_state(neib[-1]) 

*---------+---------+---------*
| 3  4  4 | 5  6  4 | 9  7  9 |
| 1  8  5 | 1  9  9 | 7  7  2 |
| 2  2  3 | 5  7  8 | 4  1  5 |
*---------+---------+---------*
| 7  2  5 | 9  1  4 | 2  4  9 |
| 2  4  9 | 5  5  2 | 5  6  9 |
| 9  3  1 | 9  8  7 | 6  7  5 |
*---------+---------+---------*
| 4  9  7 | 4  3  1 | 3  4  7 |
| 8  1  8 | 7  4  5 | 3  9  6 |
| 2  8  3 | 4  6  6 | 9  8  9 |
*---------+---------+---------*


Ya podemos empezar a percibir el desafío de resolver un Sudoku utilizando algoritmos de búsqueda local. Por cada estado, se generan muchos estados vecinos nuevos, lo que aumenta considerablemente el espacio de búsqueda y la complejidad del problema.

----
## Función de costo

Ya hemos visto que el Sudoku genera muchísimos vecinos. Podemos imaginar que el Sudoku se encuentra en un espacio de dimensión igual al número de celdas. Por ejemplo, en un Sudoku de 3x3, la dimensión sería de 81.

Para poder aplicar métodos de búsqueda local, necesitamos una forma de medir cuán lejos estamos de la solución. Es decir, necesitamos una función de costo o energía que, al pasarle un estado, nos devuelva un valor escalar. Esta función tomará como entrada las 81 celdas y devolverá un único número que indique qué tan cerca estamos de la solución.

Esta función es clave para que los algoritmos de búsqueda local funcionen correctamente. En muchos casos, las funciones de costo están predefinidas por el problema, pero en este caso debemos diseñarla desde cero. Por eso, vamos a buscar una manera de que la función devuelva su valor mínimo cuando el Sudoku esté correctamente resuelto, evitando así problemas como mínimos locales, mesetas o crestas, los cuales son comunes en espacios de alta dimensión.

El diseño elegido para la función de costo es el siguiente:

- Para cada unidad (fila, columna o caja):
    - Si un dígito no se encuentra, se suma una penalización de *0.05*.
    - Si un dígito se repite una vez, se suma una penalización de *0.05*
    - Si un dígito se repite más de una vez, se suma una penalización de *N * 0.05*, donde *N* es la cantidad de repeticiones del dígito.
- Si el estado es una solución correcta, cada fila, columna y caja contendrá exactamente una vez cada dígito del 1 al 9. En ese caso, el costo será 0, lo que corresponde al mínimo global.

Esta lógica está implementada en la función `cost_function()`, que toma un estado como entrada y devuelve el costo correspondiente.

Veamos ahora el costo del estado aleatorio que generamos:

In [20]:
print(f"El costo del estado es {cost_function(new_state)}") 

El costo del estado es 6.8999999999999995


Y comparemos con el costo de la solución:

In [21]:
print(f"El costo del estado es {cost_function(solution)}") 

El costo del estado es 0.0


¿Se te ocurre alguna otra función de costo que podrías implementar?